In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing in

In [ ]:

!pip install PyPDF2
!pip install langchain
!pip install python-dotenv


!pip install langchain-community  # Install the required community package

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.4 MB/s eta 0:00:00


In [ ]:
!pip install langchain-google-genai

In [ ]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.8 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import stat
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import traceback


load_dotenv()


api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("API key not found. Please set the GOOGLE_API_KEY environment variable.")

genai.configure(api_key=api_key)

def get_pdf_text(pdf_files):
    text = ""
    try:
        for pdf_file in pdf_files:
            pdf_reader = PdfReader(pdf_file.name)
            for page in pdf_reader.pages:
                text += page.extract_text() or ""
    except Exception as e:
        return f"Error reading PDF files: {e}"
    return text

def get_text_chunks(text):

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=15000, chunk_overlap=2000)
    chunks = text_splitter.split_text(text)
    return chunks

def get_vector_store(text_chunks):
    try:
        embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)
        vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
        vector_store.save_local("faiss_index")
    except Exception as e:
        return f"Error in creating vector store: {e}"

def get_conversational_chain():
    prompt_template = """
    Answer the question as thoroughly and detailed as possible based on the provided context. If the answer is not in
    the provided context, state clearly that "the answer is not available in the context."

    Context:
    {context}

    Question:
    {question}

    Detailed Answer:
    """

    try:
        model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.2)  # Lower temperature for more deterministic output
        prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
        chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
        return chain
    except Exception as e:
        return f"Error in setting up conversational chain: {e}"

def process_pdfs(pdf_files):
    raw_text = get_pdf_text(pdf_files)
    if "Error" in raw_text:
        return raw_text

    text_chunks = get_text_chunks(raw_text)
    if os.path.exists("faiss_index"):
        try_remove_file("faiss_index")
    error = get_vector_store(text_chunks)
    if error:
        return error
    return "Processing complete. You can now ask questions."

def user_input(user_question):
    try:
        embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)
        new_db = FAISS.load_local("faiss_index", embeddings=embeddings, allow_dangerous_deserialization=True)
        docs = new_db.similarity_search(user_question)


        print("Retrieved Documents:")
        for doc in docs:
            print(doc.page_content[:500])


        context = " ".join([doc.page_content for doc in docs])

        if not context.strip():
            return "No relevant context found in the PDF files."

        chain = get_conversational_chain()
        if isinstance(chain, str):
            return chain
        response = chain({"input_documents": docs, "question": user_question}, return_only_outputs=True)
        return response.get("output_text", "No output text found.")
    except Exception as e:
        return f"Error in answering the question: {e}"

def try_remove_file(file_path):
    try:
        os.chmod(file_path, stat.S_IWRITE)
        os.remove(file_path)
    except Exception as e:
        return f"Error deleting file: {e}"

def main_interface(pdf_files, user_question):
    if pdf_files:
        processing_message = process_pdfs(pdf_files)
        if "Error" in processing_message:
            return processing_message
    if user_question:
        response = user_input(user_question)
        return response
    return "Please upload PDF files and ask a question."


pdf_input = gr.File(label="Upload your PDF Files", file_count="multiple", file_types=[".pdf"])
question_input = gr.Textbox(label="Ask a Question from the PDF Files")
output = gr.Textbox(label="Reply")

interface = gr.Interface(fn=main_interface, inputs=[pdf_input, question_input], outputs=output, title="Chat with PDF", description="Upload PDFs and ask questions.")

if __name__ == "__main__":
    interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a6ce9b98642f1e8a0c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
